# Chains

**Sumario**

1. Introducción
   1. ¿Por qué necesitamos cadenas cuando usamos LLMs?
   2. Diferentes maneras de llamar a las cadenas
   3. ¿Cómo añadir memoria a las cadenas?
   4. ¿Cómo depurar las cadenas?
<br></br>
2. Cadenas simples
   1. `LLMChain`
   2. `TransformChain`
<br></br>
1. Cadenas secuenciales
   1. `SimpleSequentialChain`
   2. `SequentialChain`
<br></br>
2. Cadenas de enrutamiento
   1. `LLMRouterChain`


In [ ]:
import openai
import os
from langchain.llms import AzureOpenAI

# Lee la clave de API desde el archivo de configuración
with open('config.txt') as f:
    config = dict(line.strip().split('=') for line in f)

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = config.get("OPENAI_API_KEY", "")

# Nombre del despliegue en mi Azure OpenAI Studio is "Davinci003", el modelo es "text-davinci-003"
engine = "Davinci003"
model = "text-davinci-003"
openai_api_version = "2023-12-01" 

# Nombre del despliegue en mi Azure OpenAI Studio para el modelo de embeddings es "TextEmbeddingAda002"
embeddings_engine = "TextEmbeddingAda002"

max_tokens = 1000

llm = AzureOpenAI(
    azure_endpoint=openai.api_base, 
    azure_deployment=engine, 
    openai_api_key=config.get("OPENAI_API_KEY", ""), 
    openai_api_version=openai.api_version,
    # temperature=0, # Podemos poner la temperatura a 0 si queremos reducir la variabilidad de las respuestas
)
llm.openai_api_base = openai.api_base 
llm.max_tokens = max_tokens

# 1 - Introducción

Las cadenas nos permiten fusionar varios componentes en una aplicación unificada. Por ejemplo, podemos formar una cadena que incorpore la entrada del usuario, aplique un [`PromptTemplate`](https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html) para darle formato y posteriormente transfiera la respuesta formateada a un modelo de lenguaje grande (LLM, por sus siglas en inglés). Al combinar varias cadenas o integrarlas con otros componentes, podemos construir cadenas aún más intrincadas.

La cadena más simple de estas es la [`LLMChain`](https://python.langchain.com/en/latest/modules/chains/generic/llm_chain.html). Funciona de la siguiente manera:
1. Toma la entrada del usuario.
2. La pasa al primer elemento de la cadena (es decir, un [`PromptTemplate`](https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html)) para dar formato a la entrada y generar un prompt específico.
3. El prompt formateado se pasa al siguiente (y último) elemento de la cadena, es decir, un LLM.

Comenzaremos "creando" nuestro LLM, que será una instancia de GPT 3.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Creamos el template para nuestro prompt
prompt = PromptTemplate(
    input_variables=["producto"],
    template="Cual es un buen nombre para una empresa de {producto} (devuelve un único ejemplo. Nada más)"
)

# Creamos la cadena mas simple posible que nos permite interactuar con un LLM mediante un prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Corremos la cadena
print(llm_chain.run("bicicletas de montaña"))

Para evitar gastos inesperados, podemos usar la siguiente función que nos indica cuantos tokens estamos utilizando en cada llamada. Ésto es especialmente útil cuando usamos módulos mas avanzados como Agentes, los cuales pueden realizar múltiples llamadas a la API.

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'{cb.total_tokens} tokens han sido utilizados')

    return result

In [ ]:
print(count_tokens(llm_chain, "bicicletas de montaña"))

Si tenemos múltiples variables, podemos introducirla con un diccionario:

In [ ]:
prompt = PromptTemplate(
    input_variables=["producto", "numero"],
    template="Cual es un buen nombre para una empresa de {producto} (devuelve {numero} ejemplos. Nada más)",
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
query = {
    'producto': "bicicletas de montaña",
    'numero': "3"
    }
print(count_tokens(llm_chain, query))

## 1.2 - Diferentes maneras de llamar a las cadenas

Todas las clases que heredan de [`Chain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.base.Chain.html?highlight=chain#langchain.chains.base.Chain) ofrecen diferentes maneras de ser utilizadas:

### 1.2.1 - `__call__`

In [ ]:
prompt_template = "Cuentame un chiste {tipo}"
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

llm_chain(inputs={"tipo": "de miedo"})

Podemos configurarlo para que solo nos devuelva el output:

In [ ]:
llm_chain(inputs={"tipo": "de miedo"}, return_only_outputs=True)

### 1.2.1 - `run()`

Si la cadena solo devuelve una cosa (i.e., solo tiene un elemento en su `output_keys`), podemos usar el método `run`, el cual devuelve un string en vez de un diccionario.

In [ ]:
# llm_chain solo tiene un elemento en output_keys, por lo que podemos usarlo con run
llm_chain.output_keys

In [ ]:
llm_chain.run({"tipo":"de miedo"})

## 1.3 - ¿Cómo añadir memoria a las cadenas?

`Chain` admite tomar un objeto `BaseMemory` como su argumento `memory`, lo que permite que el objeto `Chain` persista datos a lo largo de múltiples llamadas. En otras palabras, convierte a `Chain` en un objeto con estado.

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversacion = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

print(conversacion.run("Responde brevemente. ¿Cuáles son los primeros 3 colores de un arcoíris? (Devuelve solo los nombres)"))
print(conversacion.run("¿Y los siguientes 4?"))

Esencialmente, `BaseMemory` define una interfaz sobre cómo LangChain almacena la memoria. Permite la lectura de datos almacenados a través del método `load_memory_variables()` y el almacenamiento de nuevos datos mediante el método `save_context`.

Veremos más sobre la memoria en las siguientes clases.

## 1.4 - ¿Cómo depurar las cadenas?

Puede ser difícil depurar un objeto `Chain` únicamente desde su salida, ya que la mayoría de los objetos `Chain` involucran una cantidad considerable de preprocesamiento de la entrada del prompt y postprocesamiento de la salida del LLM. Establecer `verbose=True` imprimirá algunos estados internos del objeto `Chain` mientras se está ejecutando.

In [ ]:
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
    verbose=True
)
conversation.run("¿Qué es ChatGPT?")

LangChain agrega un texto por defecto a las prompts cuando utilizamos ciertas cadenas como `ConversationChain` para guiar al modelo de lenguaje. Si bien estamos utilizando el español como lenguaje, el texto se encuentra por defecto en inglés, lo cual podría confundir al modelo. Más adelante, veremos cómo podemos modificar estas prompts por defecto para que se alineen con nuestro idioma y objetivos específicos.


In [ ]:
import langchain

langchain.debug = True

In [ ]:
conversation.run("¿Qué es ChatGPT?")

In [ ]:
langchain.debug = False

# 2 - Cadenas simples

## 2.1 - `LLMChain`

<table>
    <tr>
        <td><img src="images_3_1/llm_chain.png" width="700"/></td>
    </tr>
</table>

Hemos visto ejemplos de la misma más arriba

## 2.2 - `TransformChain`

<table>
    <tr>
        <td><img src="images_3_1/transform_chain.png" width="700"/></td>
    </tr>
</table>

Como ejemplo, si hemos lidiado con textos de entrada desordenados, lo cual podría resultar en un costo adicional debido a la tarificación por tokens en APIs como OpenAI, podemos crear una función personalizada para limpiar los espacios en nuestros textos:

In [ ]:
import re
from langchain.chains import TransformChain

def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    
    # reemplaza múltiples saltos de línea y múltiples espacios con uno solo
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)
clean_extra_spaces_chain.run('Un texto aleatorio  con   espaciado irregular.\n\n\n     Otro aquí   también.')

Este tipo de cadena no se suele usar por si sola, sino que suele ser combinada con otras cadenas como `LLMChain`. Ahora veremos como podemos combinar diferentes cadenas.

# 3 - Cadenas secuenciales

La idea detrás de las cadenas secuenciales es combinar varias cadenas donde **la salida de una cadena es la entrada de la siguiente**. Hay dos tipos de cadenas secuenciales:

* `SimpleSequentialChain`: Una sola entrada/salida
* `SequentialChain`: Múltiples entradas/salidas

## 3.1 - `SimpleSequentialChain`

<table>
    <tr>
        <td><img src="images_3_1/simple_sequential_chain.png" width="700"/></td>
    </tr>
</table>

In [ ]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate

# Primera cadena
first_prompt = ChatPromptTemplate.from_template(
    "¿Cuál es el mejor nombre para describir a una empresa que fabrica {producto} (devuelve solo un ejemplo, nada más)?"
)
first_chain = LLMChain(llm=llm, prompt=first_prompt)

# Segunda cadena
second_prompt = ChatPromptTemplate.from_template(
    "Escribe una descripción de 20 palabras en español para la siguiente empresa: {nombre_empresa}"
)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
simple_seq_chain = SimpleSequentialChain(chains=[first_chain, second_chain], verbose=True)
print(count_tokens(simple_seq_chain, "bicicletas de montaña"))

## 3.2 - `SequentialChain`

`SimpleSequentialChain` funciona bien cuando tenemos una sola entrada y una sola salida. Pero, ¿qué sucede cuando tenemos múltiples entradas y salidas?

A modo de ejemplo, vamos a realizar los siguientes pasos:
* Crear dos breves historias utilizando dos cadenas independientes, una en español y otra en inglés. Podríamos emplear dos modelos diferentes, por ejemplo.
* Traducir la historia en inglés a español.
* Evaluar cual de las dos historias es más entretenida.

<table>
    <tr>
        <td><img src="images_3_1/sequential_chain.png" width="800"/></td>
    </tr>
</table>

### 3.2.1 - Crear breves historias

In [ ]:
# Historia en español
spanish_story_prompt = ChatPromptTemplate.from_template(
    "Crea una breve cuento de 50 palabras sobre un {protagonista} en español:"
)
spanish_story_chain = LLMChain(llm=llm, prompt=spanish_story_prompt, output_key="spanish_story")

# Historia en inglés
english_story_prompt = ChatPromptTemplate.from_template(
    "Crea una breve cuento de 50 palabras sobre un {protagonista} en inglés:"
)
english_story_chain = LLMChain(llm=llm, prompt=english_story_prompt, output_key="english_story")

### 3.2.2 - Traducir la historia en inglés a español

In [ ]:
# Recibe como input la salida de la "english_story_chain"
translate_prompt = ChatPromptTemplate.from_template(
    "Traduce el siguiente cuento al español:"
    "\n\n{english_story}"
)
translate_chain = LLMChain(llm=llm, prompt=translate_prompt, output_key="translated_story")

### 3.2.3 - Evaluar cual de las dos historias es más entretenida

In [ ]:
# Recibe como inputs los dos cuentos en español (uno original y el otro la traduccion del de ingles)
evaluate_prompt = ChatPromptTemplate.from_template(
    "Otorga una puntuacion entre 1 y 10 a las siguientes cuentos segun su nivel de entretenimiento"
    "\n\nCuento 1: {spanish_story}\n\nCuento 2: {translated_story}"
)
evaluate_chain = LLMChain(llm=llm, prompt=evaluate_prompt, output_key="puntuacion")

### 3.2.4 - Crear la `SequentialChain`

In [ ]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[spanish_story_chain, english_story_chain, translate_chain, evaluate_chain],
    input_variables=["protagonista"],
    output_variables=["spanish_story", "english_story", "translated_story", "puntuacion"],
    verbose=True,
)

# "Crea una breve cuento de 50 palabras sobre un {protagonista}"
protagonista = "gato con botas"
overall_chain(protagonista)

# 4 - Cadenas de enrutamiento

## 3.3 - `LLMRouterChain`

Básicamente, generamos una cadena específica respaldada por un LLM que redirige a la subcadena adecuada al leer la pregunta y las descripciones de la plantilla de indicaciones.

<table>
    <tr>
        <td><img src="images_3_1/router_chain.png" width="700"/></td>
    </tr>
</table>

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

### 3.3.1 - Crear templates

Asumiendo que el la cadena de enrutamiento funciona correctamente, pasaria a utilizar el template correspondiente para "influenciar" al LLM a comportarse lo mas cercano a la pregunta posible. Por ejemplo, si la pregunta es sobre matematicas, le ponemos un encabezado diciendo que sabe de matematicas ya que se ha mostrado experimentalmente que mejora la calidad de los resultados.

In [ ]:
physics_template = """Eres un profesor de biología muy inteligente. \
Eres excelente para responder preguntas sobre biología de manera concisa \
y fácil de entender. \
Cuando no sabes la respuesta a una pregunta, admites \
que no lo sabes.

Aquí tienes una pregunta:
{input}"""


math_template = """Eres un matemático muy hábil. \
Eres excelente para responder preguntas de matemáticas. \
Eres tan bueno porque puedes descomponer \
problemas difíciles en sus partes componentes, \
responder a esas partes y luego unirlas \
para responder a la pregunta más amplia.

Aquí tienes una pregunta:
{input}"""

history_template = """Eres un historiador muy competente. \
Tienes un excelente conocimiento y comprensión de las personas, \
eventos y contextos en diferentes períodos históricos. \
Tienes la capacidad de pensar, reflexionar, debatir, discutir y \
evaluar el pasado. Tienes respeto por la evidencia histórica \
y la habilidad de utilizarla para respaldar tus explicaciones \
y juicios.

Aquí tienes una pregunta:
{input}"""

### 3.3.2 - Información sobre los templates para el router

Para que el router redirija correctamente tiene que tener una descripcion de los diferentes templates:

In [ ]:
prompt_infos = [
    {
        "nombre": "Biología", 
        "descripcion": "Bueno para responder preguntas sobre biología", 
        "prompt_template": physics_template
    },
    {
        "nombre": "Matemáticas", 
        "descripcion": "Bueno para responder preguntas sobre matemáticas", 
        "prompt_template": math_template
    },
    {
        "nombre": "Historia", 
        "descripcion": "Bueno para responder preguntas sobre historia", 
        "prompt_template": history_template
    },
]

### 3.3.4 - Cadenas específicas

Vamos a crear una cadena especifica para cada uno de los templates. Al igual que en casos anteriores, estamos usando siempre el mismo modelo (GPT-3) pero podriamos usar modelos diferentes para cada uno de los casos. Por ejemplo si trabajasemos con diferentes tipos de inputs (e.g., imagenes, formulas matematicas, etc.) donde tuviera sentido usar modelos especializados.

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["nombre"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['nombre']}: {p['descripcion']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
destination_chains

En caso de que no tenga claro que va con ninguna de los templates disponibles, puede utilizar el modelo por defecto sin ningun tipo de prompt especifico (e.g., matematicas, biologia, historia, etc.):

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

### 3.3.5 - Crear `LLMRouterChain`

Ya hemos creado las cadenas espec´ficias y les hemos asignado una descripción para que el modelo de enrutamiento sepa cual es la mejor cadena para cada caso.

Ahora, tenemos que crear la cadena de enrutamiento. Sin embargo, tenemos un pequeño problema, y es que el template por defecto que utiliza `RouterChain` se encuentra en inglés. 

Ante esto tenemos dos posibilidades:

* Trabajamos en inglés y traducimos los textos con una cadena secuencial como hemos visto anteriormente.
* Modificamos el template para que el modelo de enrutamiento sepa que el texto le viene en español.

In [ ]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

print(MULTI_PROMPT_ROUTER_TEMPLATE)

In [ ]:
custom_router_template = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

IMPORTANT: Both inputs and candidate prompts are in Spanish

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
custom_router_template = custom_router_template.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=custom_router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

### 3.3.6 - Combinar cadenas

Para poder combinar las cadenas necesitamos una ruta que agrupa todos los componentes anteriores que es la `MultiPromptChain`

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True
                        )

In [ ]:
chain.run("¿Qué dice el teorema de Fermat?")

In [ ]:
langchain.debug = True
chain.run("¿En que año cayó el muro de Berlin?")

# Siguiente

En la siguiente lección, veremos como podemos añadir memoria a nuestras cadenas para que el modelo recuerde las interacciones pasadas y/o datos relevantes.